## References

- https://medium.com/@sahiljadon/rate-limiting-using-redis-lists-and-sorted-sets-9b42bc192222
- https://redislabs.com/redis-best-practices/basic-rate-limiting/
- https://www.binpress.com/rate-limiting-with-redis-1/
- https://blog.callr.tech/rate-limiting-for-distributed-systems-with-redis-and-lua/
- https://brandur.org/redis-cluster
- https://engineering.classdojo.com/blog/2015/02/06/rolling-rate-limiter/

In [1]:
import time

import redis

In [2]:
r = redis.Redis(decode_responses=True)

In [3]:
r.ping()

True

In [4]:
r.set("hello", "world")

True

In [6]:
# Basic lua script.
lua = "return redis.call('GET', KEYS[1])"
r.eval(lua, 1, "hello")

'world'

In [7]:
# Alternative with register_script method.
lua = """
local value = redis.call('GET', KEYS[1])
value = tonumber(value)
return value * ARGV[1]"""
multiply = r.register_script(lua)

r.set("foo", 2)
multiply(keys=["foo"], args=[2])

4

## Allow only N API requests per minute

In [17]:
class RateLimiter:
    def __init__(self, conn, limit=5):
        self.conn = conn
        self.nkeys = 1
        self.limit = limit
        self.script = """
            if redis.call('EXISTS', KEYS[1]) == 0 then
                redis.call('SETEX', KEYS[1], 60, 0)
            end
            redis.call('INCR', KEYS[1])
            if tonumber(redis.call('GET', KEYS[1])) <= tonumber(ARGV[1]) then
                return 'ok'
            else
                return 'limit exceeded'
            end
        """

    def allow(self, ip):
        return self.conn.eval(self.script, self.nkeys, ip, self.limit) == "ok"

In [18]:
ratelimit = RateLimiter(r)

In [24]:
ratelimit.allow(1)

False

In [55]:
r.get("1")

'1'

## Allow only N API requests per minute on a running window

In the previous redis version, passing `time` to the script is always the recommended approach, as mentioned [here](https://redis.io/docs/manual/programmability/eval-intro/#:~:text=Acts%20such%20as%20using%20the%20system%20time%2C%20calling%20Redis%20commands%20that%20return%20random%20values%20(e.g.%2C%20RANDOMKEY)%2C%20or%20using%20Lua%27s%20random%20number%20generator%2C%20could%20result%20in%20scripts%20that%20will%20not%20evaluate%20consistently.). Call to `redis.call("TIME")` is not recommended.

However, in the newer version, you can do that.

In [55]:
# Returns seconds
int(time.time())

1673341712

In [62]:
class RateLimiter:
    def __init__(self, conn, n=5):
        self.conn = conn
        self.n = n
        self.lua = self.conn.register_script(
            f"""
            -- ARGV[1]: The limit.
            -- ARGV[2]: The current timestamp in seconds.
            -- KEYS[1]: The key to rate limit, e.g. clientIP + userID/sessionID
            local count = tonumber(redis.call('LLEN', KEYS[1]))
            if count < tonumber(ARGV[1]) then
                redis.call('LPUSH', KEYS[1], now)
                return 'ok'
            else
                local now = redis.call('TIME')[1]
                local time = tonumber(redis.call('LINDEX', KEYS[1], -1))
                if now - time < 60 then
                    return 'limit exceeded'
                else
                    -- Push the timestamp to the list.
                    redis.call('LPUSH', KEYS[1], now)
                    
                    -- Remove previous item in the list.
                    redis.call('RPOP', KEYS[1])
                    return 'ok'
                end
            end
        """
        )

    def allow(self, ip):
        return self.lua(keys=[ip], args=[self.n]) == "ok"

In [63]:
ratelimit = RateLimiter(r, 10)

In [75]:
ratelimit.allow(1)

False

In [76]:
r.llen("1")

10

## Rate limiting using sorted set

In [12]:
class RateLimiter:
    def __init__(self, conn, n=5):
        self.conn = conn
        self.n = n
        self.lua = self.conn.register_script(
            f"""
            -- ARGV[1]: The current limit.
            -- KEYS[1]: The key to rate limit, e.g. clientIP + userID/sessionID.
            local limit = tonumber(ARGV[1])
            local now = redis.call('TIME')
            -- The first argument is seconds, the second is microseconds.
            -- Convert them to microseconds.
            local now_ms = math.floor(now[1] * 1000 + now[2] / 1000)
            
            -- Delete all keys that are older than 1 minute ago.
            redis.call('ZREMRANGEBYSCORE', KEYS[1], 0, now_ms - 60*1000)
            
            -- Find the number of remaining tokens left. 
            if tonumber(redis.call('ZCARD', KEYS[1])) < limit then
                redis.call('ZADD', KEYS[1], now_ms, now_ms)
                return 'ok'
            else
                return 'limit exceeded'
            end
        """
        )

    def allow(self, ip):
        # We need millisecond precisions - else the seconds will be counted as 1 item in the sorted set.
        return self.lua(keys=[ip], args=[self.n]) == "ok"

In [13]:
ratelimit = RateLimiter(r)
for i in range(6):
    print(ratelimit.allow("0.0.0.0"))

True
True
True
True
True
False


In [83]:
r.zrange("0.0.0.0", 0, -1, withscores=True)

[('1673342160207', 1673342160207.0),
 ('1673342160221', 1673342160221.0),
 ('1673342160223', 1673342160223.0),
 ('1673342160236', 1673342160236.0),
 ('1673342160247', 1673342160247.0)]

# Allow 5 req per minute, but in future time

In [47]:
class RateLimiter:
    def __init__(self, conn, n=5):
        self.conn = conn
        self.n = n
        self.period = 60 / n
        self.lua = self.conn.register_script(
            f"""
            -- KEYS[1]: The key to rate limit, e.g. clientIP + userID/sessionID.
            -- ARGV[1]: The period before the next call in seconds.
            local key = KEYS[1]
            local period = tonumber(ARGV[1])
            
            -- The first argument is seconds, the second is microseconds.
            -- Convert everything to microsends.
            local now = redis.call('TIME')
            local now_ms = now[1] * 10^6 + now[2]
            
            local future = redis.call('GET', key)
            if future ~= nil then
                future = tonumber(future)
            else
                future = 0
            end

            if now_ms < future then
                return 'limit exceeded'
            else
                redis.call('SET', key, now_ms + (period * 10^6))
                return 'ok'
            end
        """
        )

    def allow(self, ip):
        return self.lua(keys=[ip], args=[self.period]) == "ok"

In [51]:
ratelimit = RateLimiter(r)
for i in range(6):
    print(ratelimit.allow("0.0.0.0"))

False
False
False
False
False
False


In [20]:
# After making the first call, we compute the time where the next possible call can be made.
future = 0
n = 5
period = 1 / n  # 5 requests per second.

start = int(time.time_ns() / 1e6)

for i in range(10):
    now = int(time.time_ns() / 1e6)  # ms
    if now < future:
        print("limited", end=": ")
    else:
        future = now + period * 1000
        print("okay", end=": ")
    time.sleep(0.1)
    print("elapsed", now - start)

okay: elapsed 0
limited: elapsed 133
okay: elapsed 241
limited: elapsed 346
okay: elapsed 447
limited: elapsed 552
okay: elapsed 656
limited: elapsed 760
okay: elapsed 862
limited: elapsed 968


In [26]:
def time_ms():
    return time.time_ns() // 1e6

In [29]:
time_ms()

1673345783208.0

In [49]:
import math

# In this implementation, we store the future time the call can be made, as well as the counter
# future = t_future * 1000 + n_counter

future = 0
t = 1
n = 5  # Also 5 request per second, but with smoothing.
period = t / n

# Microseconds, we will store the present counter in the future time.
# The counter n cannot be more than 1000 (there is an option to use nanoseconds too)
start = time_ms() * 1000


for i in range(20):
    if i == 0:
        time.sleep(0.5)
    now = time_ms() * 1000  # microseconds
    count = future % 1000

    # At every time interval, we can only make certain number of calls.
    rem = n - ((future - now) // (period * 1e6))
    print("count", count, "rem", rem, end=", ")

    # Already exceeded the rate-limit period, can make new calls.
    if now > future:
        # The future time is the time taken to make the 5 requests, which is 1 second
        # We add the counter 1 at the end to indicate a call has been consumed.
        future = now + (t * 1e6) + 1
        print("done okay", end=": ")
    elif count < rem:
        print("okay", end=": ")
        future += 1
    else:
        print("limited", end=": ")
    time.sleep(0.1)
    print("elapsed", (now - start) / 1e3)

count 0 rem 8366733669.0, done okay: elapsed 505.0
count 1.0 rem 1.0, limited: elapsed 609.0
count 1.0 rem 2.0, okay: elapsed 714.0
count 2.0 rem 2.0, limited: elapsed 817.0
count 2.0 rem 3.0, okay: elapsed 921.0
count 3.0 rem 3.0, limited: elapsed 1026.0
count 3.0 rem 4.0, okay: elapsed 1127.0
count 4.0 rem 4.0, limited: elapsed 1229.0
count 4.0 rem 5.0, okay: elapsed 1331.0
count 5.0 rem 5.0, limited: elapsed 1436.0
count 5.0 rem 6.0, done okay: elapsed 1540.0
count 1.0 rem 1.0, limited: elapsed 1643.0
count 1.0 rem 2.0, okay: elapsed 1747.0
count 2.0 rem 2.0, limited: elapsed 1851.0
count 2.0 rem 3.0, okay: elapsed 1959.0
count 3.0 rem 3.0, limited: elapsed 2059.0
count 3.0 rem 4.0, okay: elapsed 2160.0
count 4.0 rem 4.0, limited: elapsed 2264.0
count 4.0 rem 5.0, okay: elapsed 2370.0
count 5.0 rem 5.0, limited: elapsed 2474.0
